In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import glob
import shutil
import hashlib
from collections import OrderedDict
import pandas as pd
import time
import pickle
from utils import datasets

In [ ]:

HEADER_ROW=['bag', 'time_sec', 'time_nsec', 'GPS', 'v_ego', 'key_event', 'frame_name']
frames_dir = '/media/dean/deans_data/construction_frames/'
bag_dirs = os.listdir(frames_dir)
save_dir = os.path.join(frames_dir,'kache_set')
os.makedirs(save_dir, 0o755 , exist_ok = True )

img_data = []
lookup_table = {}

for bag in bag_dirs:
    if os.path.join(frames_dir,bag) != save_dir:
        img_dict = {}
        bag_path = os.path.join(frames_dir, bag)
        img_dict['bag_path'] = bag_path
        img_dict['frames'] = []
        img_dict['csv_path'] = os.path.join(bag_path, 'frames.csv')
        csv_logger = pd.read_csv(img_dict['csv_path'], names=HEADER_ROW, skiprows=1)

        # Get all images
        imgs = glob.glob(os.path.join(bag_path,'**/', '*.jpg'))

        for img in imgs:
            d = {}
            d['frame_path'] = img
            new_path = os.path.split(os.path.split(os.path.split(img)[0])[0])[1].rstrip('/')+'-'+os.path.split(img)[1]
            hash_object = hashlib.sha1(str.encode(new_path))
            hex_dig = hash_object.hexdigest()
            #shutil.copyfile(img, os.path.join(save_dir, str(hex_dig)+'.jpg'))
            d['dataset_path'] = os.path.join(save_dir, str(hex_dig)+'.jpg')
            d['hash_key'] = hex_dig
            d['hash_val'] = new_path
            lookup_table[d['hash_key']] = d['hash_val'] 
            old_path = os.path.join(bag_path, 'frames', d['frame_path'].split('-')[-1])

            for log in [x for x in csv_logger.as_matrix() if os.path.join(bag_path, 'frames', x[6].rstrip('/')) == old_path.rstrip('/')]:
                d['bag_name'] = log[0]
                d['time_sec'] = log[1]
                d['time_readable'] = time.ctime(int(log[1]))

                d['time_nsec'] = log[2]
                d['GPS'] = log[3]
                d['v-ego'] = log[4]
                                    
                d['key_event'] = log[5]
                d['frame'] = log[6]

            if d.get('frame', None): img_dict['frames'].append(d)
        if img_dict['frames']: img_data.append((bag_path,img_dict))

pickle_file = '/media/dean/datastore/datasets/kache_ai/frames/kacheai_logs.pickle'
pickle_dict = {'img_data':img_data,'lookup_table':lookup_table}
print('Saving to Pickle File:', pickle_file)
with open(pickle_file,"wb") as pickle_out:
    pickle.dump(pickle_dict, pickle_out)



In [ ]:
print(img_data[5])

In [ ]:


WORKING_DIR = '/media/dean/datastore/datasets/kache_ai'
kache_set = datasets.DataFormatter(annotations_list = '/media/dean/datastore/datasets/kache_ai/frames/kacheai_logs.pickle', image_list = '/media/dean/datastore/datasets/kache_ai/frames/', input_format = datasets.Format.kache,
                                 output_path = os.path.join(WORKING_DIR,'scalabel_files'),
                                 trainer_prefix = 'COCO_train2014_0000', 
                                 s3_bucket = 'kache-scalabel/kache_ai/frames/', check_s3 = False)

Initializing Annotation Pipeline via Darknet ... \o/ \o/


In [ ]:
kache_set.export(format = datasets.Format.scalabel)

In [ ]:
# 86469: 4.864921, 4.839282 avg loss, 0.000080 rate, 13.939101 seconds, 177088512 images
